# Yelp Review
## Business Problem
In today’s information era, even non technological
savvy consumers can quickly discover information
about a business. This is great for customers, as it
allows them to identify the “snake oil salesmen” (this
is a metaphor that was coined for individuals who sold
fraudulent goods). The motivation behind this project
is to develop a tool that can show businesses, in this
case restaurants, what aspects of their business are
leading to their reviews without them having to crawl
through potentially hundreds of text data. Is it their
price? Their food? Their service? Businesses that know
this information can get ahead of the competition by
improving the areas that lead to less positive reviews.
For example, a restaurant that knows they possess four
stars on Yelp, along with a breakdown of which
aspects of their establishment contributed to these four
stars might see that they lose marks on their service.
This may encourage management to retrain
waiters/waitresses in an effort to improve their service

score and thus their overall Yelp score. This prospect
would be extremely useful to restaurants because when
was the last time the reader went to a restaurant
without reading some online reviews beforehand?

## The Data
The data, as discussed above, is text review data. In this re-enforcement learning stage, the data set comes from two locations. One set is a snapshot of about 10,000 records from a larger database where the trained model will be deployed on. This snapshot was manually coded with one of the following categories
* Food#Postive
* Food#Negative
* Price#Postive
* Price#Negative
* Quality#Positive
* Quality#Negative
* Restaurant#Positive
* Restaurant#Negative
* Location#Postive
* Location#Negative
* Service#Positive
* Service#Negative
This was done for validation purposes. The other 1,600 records come from a Yelp Kaggle competition. The categories were coded to reflect categories above.

In [3]:
# Step 1 Read in data
# Step 2 Preprocess text data
# Step 3 Word Embedding
# Step 4 Deep Learning

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
stop = stopwords.words('english')
import re
# importing keras packages
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences


from keras.layers import Flatten
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from keras.models import Sequential #### required layer in our LSTM network
from keras.layers import Dense #### required layer in our LSTM network
 #### required layer in our LSTM network
from keras.layers.embeddings import Embedding #### required layer in our LSTM network
from keras.preprocessing import sequence #### Packaged preprocessing step in Keras
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
yelp=pd.read_excel('Sample_Yelp_Ver2.xlsx')

In [3]:
print(yelp.head())

            category                                               text
0      FOOD#POSITIVE  There are so many options to choose from, what...
1  LOCATION#NEGATIVE  To save a little time, we opted to sit outside...
2   QUALITY#NEGATIVE     MISSES - Risotto: one bite was enough for me."
3      FOOD#POSITIVE  I really loved the different and inovated touc...
4  LOCATION#POSITIVE  The view of the Bellagio fountain from this pl...


## Text Analytics
Text analytics means examining text that was written by, or about, customers. You find patterns and topics of interest, and then take practical action based on what you learn.

Text analytics can be performed manually, but it is an inefficient process. Therefore, text analytics software has been created that uses text mining and natural language processing algorithms to find meaning in huge amounts of text, which is the attempt of this project.

### PreProcessing
Unlike regular data preprocessing steps, text mining requires an alternate approach. The end goal is more or less the same, a normalised data set that is numbers only containing less noise. 

#### Lower Case 
This is a step that helps put the text on an equal footing. The most simple way to explain it is as follows: One wouldn't count *Everything* and *everything* as two separate words.

In [4]:
# PreProcessing
#step 1 lower case
#step 2 punctuation
#step 3 stop word
#step 4 common word removal
#step 5 rare word removal
#step 6 token
#step 7 stemming
#step 8 lemma

In [5]:
#step 1
yelp['lower'] = yelp.text.apply(lambda x: " ".join(x.lower() for x in x.split()))
yelp.lower.head()

0    there are so many options to choose from, what...
1    to save a little time, we opted to sit outside...
2       misses - risotto: one bite was enough for me."
3    i really loved the different and inovated touc...
4    the view of the bellagio fountain from this pl...
Name: lower, dtype: object

#### Removing Punctuation
Again, this is just to keep things equal and prevent items like full stops from adding noise

In [6]:
#step 2
from nltk.tokenize import RegexpTokenizer
reg_tok = RegexpTokenizer(r'\w+')#+ is one or more
yelp['no_punc'] = yelp['lower'].apply(lambda x: ' '.join(reg_tok.tokenize(x)))
yelp.no_punc.head()

0    there are so many options to choose from whate...
1    to save a little time we opted to sit outside ...
2            misses risotto one bite was enough for me
3    i really loved the different and inovated touc...
4    the view of the bellagio fountain from this pl...
Name: no_punc, dtype: object

#### Stop Words
Stop words are those words which are filtered out before further processing of text, since these words contribute little to overall meaning, given that they are generally the most common words in a language. For instance, "the," "and," and "a," while all required words in a particular passage, don't generally contribute greatly to one's understanding of content. 

In [7]:
#step 3
yelp['no_stop'] = yelp['no_punc'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
yelp.no_stop.head()

0    many options choose whatever like good chance ...
1    save little time opted sit outside 110 degree ...
2                       misses risotto one bite enough
3    really loved different inovated touch cheff gi...
4             view bellagio fountain place really nice
Name: no_stop, dtype: object

#### Common and Rare Words
Removing the most common and the rarest words often can add a small amount of value to your model. This is because words like "good" or "food", often don't add a specific value to a model.

In [8]:
freq = pd.Series(' '.join(yelp['no_stop']).split()).value_counts()[:20]#combining all rows and then splitting and converitign and value count
freq
#looking at these, we actually want to keep them so no need to carry out this step

good          42
buffet        39
food          30
great         27
one           21
service       20
vegas         20
go            19
place         19
really        19
restaurant    18
time          17
best          16
like          15
people        14
want          14
get           13
bellagio      13
got           13
try           13
dtype: int64

In [9]:
#step 4
rare = pd.Series(' '.join(yelp['no_stop']).split()).value_counts()[-600:]


In [10]:
#step 5
rare = list(rare.index)
yelp['no_rare'] = yelp['no_stop'].apply(lambda x: " ".join(x for x in x.split() if x not in rare))


In [11]:
# just did for note
from textblob import TextBlob
# not really doing that for tutorial, this is just demo of it
yelp['no_stop'][:5].apply(lambda x: str(TextBlob(x).correct()))

0    many option choose whatever like good chance f...
1    save little time often sit outside 110 degree ...
2                       misses risotto one bite enough
3    really loved different invited touch chef give...
4             view bellagio fountain place really nice
Name: no_stop, dtype: object

#### Tokenising
Tokenization is a step which splits longer strings of text into smaller pieces, or tokens. Larger chunks of text can be tokenized into sentences, sentences can be tokenized into words, etc. Further processing is generally performed after a piece of text has been appropriately tokenized. Tokenization is also referred to as text segmentation or lexical analysis. Sometimes segmentation is used to refer to the breakdown of a large chunk of text into pieces larger than words (e.g. paragraphs or sentences), while tokenization is reserved for the breakdown process which results exclusively in words.

In [12]:
#step 6
from nltk.tokenize.treebank import TreebankWordTokenizer
_word_tokenize = TreebankWordTokenizer()
yelp['token'] = yelp['no_rare'].apply(lambda x: ' '.join(_word_tokenize.tokenize(x)))
yelp.token.head()

0         many options choose like good chance finding
1    save little time opted sit outside 110 vegas l...
2                                    misses one enough
3                          really loved different food
4             view bellagio fountain place really nice
Name: token, dtype: object

#### Stemming
Stemming is the process of eliminating affixes (suffixed, prefixes, infixes, circumfixes) from a word in order to obtain a word stem. For example, running -> run

In [13]:
#step 7
from nltk.stem.snowball import SnowballStemmer
st = SnowballStemmer("english")
yelp['stemed']=yelp['token'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
yelp.stemed.head()

0               mani option choos like good chanc find
1    save littl time opt sit outsid 110 vega littl ...
2                                      miss one enough
3                              realli love differ food
4             view bellagio fountain place realli nice
Name: stemed, dtype: object

Lemmatization is related to stemming, differing in that lemmatization is able to capture canonical forms based on a word's lemma.

In [14]:
#step 8
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
yelp['lemma']=yelp['stemed'].apply(lambda x: " ".join([wordnet_lemmatizer.lemmatize(word) for word in x.split()]))
yelp.lemma.head(20)

0                mani option choos like good chanc find
1     save littl time opt sit outsid 110 vega littl ...
2                                       miss one enough
3                               realli love differ food
4              view bellagio fountain place realli nice
5     realli quit simpl differ name peopl outsid yel...
6                  look beef like crap check make place
7     good view fountain bellagio even sit insid lik...
8                          would high recommend restaur
9                  disappoint unimpress present qualiti
10                       give 4 star minus 1 locat nois
11                               bad locat especi night
12                 price cheap servic cool place open 7
13                  server cool quick pace attitud meal
14    want tri someth tri buffet bellagio pleasant s...
15                                          place clean
16                          staff came clear tabl check
17    per person breakfast brunch thought buffet

### Word Embedding 
Word embedding is the collective name for a set of language modeling and feature learning techniques in natural language processing (NLP) where words or phrases from the vocabulary are mapped to vectors of real numbers. Conceptually it involves a mathematical embedding from a space with one dimension per word to a continuous vector space with a much lower dimension.

Methods to generate this mapping include neural networks,[1] dimensionality reduction on the word co-occurrence matrix, probabilistic models,explainable knowledge base method,and explicit representation in terms of the context in which words appear. They are needed as computers do not understand words, and in laymans terms convert words to numbers.

In [15]:
# Prepping the Word Embedding by getting dictionary length and max sentence length

yelp.lemma.str.len().max()

143

In [16]:
from collections import Counter

count=Counter(" ".join(yelp.lemma).split(" ")).items()
# print(sorted(count))

In [17]:
#length of dictionary
len(count)


546

In [18]:
#longest sentence
print(max(yelp.lemma, key=len))

go back vega 18 day probabl eat least 3 time entir staff girl front manag busser chef alway incred one brought told even want burger go dessert


In [19]:
# embeddings = tf.Variable(
#     tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
# embed = tf.nn.embedding_lookup(embeddings, train_inputs)

In [20]:

# define documents

# define class labels
encoder = LabelEncoder()
encoder.fit(yelp.category)
encoded_Y = encoder.transform(yelp.category)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

# integer encode the documents
vocab_size = 6000
encoded_docs = [one_hot(d, vocab_size) for d in yelp.lemma]
#print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 130
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)
# define the model
model = Sequential()

model.add(Embedding(vocab_size, 32, input_length=max_length))
model.add(Flatten())
model.add(Dense(12, activation='sigmoid'))
#model.add(Dropout(0.3))
#model.add(Dense(12, activation='sigmoid'))
#model.add(Dropout(0.3))
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
X_train, X_test, y_train, y_test = train_test_split(padded_docs,dummy_y,test_size=0.2)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30, batch_size=64)
# evaluate the model



[[3485 5715 3871 ...    0    0    0]
 [ 362 1220 3682 ...    0    0    0]
 [1321 4691 1402 ...    0    0    0]
 ...
 [1924 2579 3050 ...    0    0    0]
 [4621  593 4812 ...    0    0    0]
 [1484 3643 2692 ...    0    0    0]]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 130, 32)           192000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 4160)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 12)                49932     
Total params: 241,932
Trainable params: 241,932
Non-trainable params: 0
_________________________________________________________________
None
Train on 252 samples, validate on 63 samples
Epoch 1/30
252/252 [==============================] - 2s 8ms/step - loss: 2.4616 - acc: 0.1111 - val_loss: 2.4016 - val_ac

In [21]:
encoder = LabelEncoder()
encoder.fit(yelp.category)
encoded_Y = encoder.transform(yelp.category)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

# integer encode the documents
vocab_size = 6000
encoded_docs = [one_hot(d, vocab_size) for d in yelp.lemma]
#print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 130
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)
# define the model
model = Sequential()

model.add(Embedding(vocab_size, 32, input_length=max_length))
model.add(Flatten())
model.add(Dense(12, activation='sigmoid'))
#model.add(Dropout(0.3))
#model.add(Dense(12, activation='sigmoid'))
#model.add(Dropout(0.3))
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
X_train, X_test, y_train, y_test = train_test_split(padded_docs,dummy_y,test_size=0.2)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=64)
# evaluate the model

[[3485 5715 3871 ...    0    0    0]
 [ 362 1220 3682 ...    0    0    0]
 [1321 4691 1402 ...    0    0    0]
 ...
 [1924 2579 3050 ...    0    0    0]
 [4621  593 4812 ...    0    0    0]
 [1484 3643 2692 ...    0    0    0]]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 130, 32)           192000    
_________________________________________________________________
flatten_2 (Flatten)          (None, 4160)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 12)                49932     
Total params: 241,932
Trainable params: 241,932
Non-trainable params: 0
_________________________________________________________________
None
Train on 252 samples, validate on 63 samples
Epoch 1/100
252/252 [==============================] - 0s 1ms/step - loss: 2.4686 - acc: 0.0952 - val_loss: 2.4222 - val_a

Epoch 54/100
252/252 [==============================] - 0s 198us/step - loss: 0.4954 - acc: 0.9603 - val_loss: 2.3306 - val_acc: 0.2222
Epoch 55/100
252/252 [==============================] - 0s 208us/step - loss: 0.4724 - acc: 0.9563 - val_loss: 2.3237 - val_acc: 0.2540
Epoch 56/100
252/252 [==============================] - 0s 200us/step - loss: 0.4530 - acc: 0.9643 - val_loss: 2.3365 - val_acc: 0.2381
Epoch 57/100
252/252 [==============================] - 0s 228us/step - loss: 0.4325 - acc: 0.9762 - val_loss: 2.3328 - val_acc: 0.2063
Epoch 58/100
252/252 [==============================] - 0s 289us/step - loss: 0.4140 - acc: 0.9603 - val_loss: 2.3261 - val_acc: 0.2222
Epoch 59/100
252/252 [==============================] - 0s 309us/step - loss: 0.3966 - acc: 0.9603 - val_loss: 2.3153 - val_acc: 0.2540
Epoch 60/100
252/252 [==============================] - 0s 272us/step - loss: 0.3808 - acc: 0.9683 - val_loss: 2.3355 - val_acc: 0.2540
Epoch 61/100
252/252 [==========================

In [23]:
encoder = LabelEncoder()
encoder.fit(yelp.category)
encoded_Y = encoder.transform(yelp.category)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
from keras.layers import Dropout
# integer encode the documents
vocab_size = 6000
encoded_docs = [one_hot(d, vocab_size) for d in yelp.lemma]
#print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 130
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)
# define the model
model = Sequential()

model.add(Embedding(vocab_size, 32, input_length=max_length))
model.add(Flatten())
model.add(Dense(12, activation='sigmoid'))
model.add(Dropout(0.3))
#model.add(Dense(12, activation='sigmoid'))
#model.add(Dropout(0.3))
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
X_train, X_test, y_train, y_test = train_test_split(padded_docs,dummy_y,test_size=0.2)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=64)
# evaluate the model

[[3485 5715 3871 ...    0    0    0]
 [ 362 1220 3682 ...    0    0    0]
 [1321 4691 1402 ...    0    0    0]
 ...
 [1924 2579 3050 ...    0    0    0]
 [4621  593 4812 ...    0    0    0]
 [1484 3643 2692 ...    0    0    0]]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 130, 32)           192000    
_________________________________________________________________
flatten_3 (Flatten)          (None, 4160)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 12)                49932     
_________________________________________________________________
dropout_1 (Dropout)          (None, 12)                0         
Total params: 241,932
Trainable params: 241,932
Non-trainable params: 0
_________________________________________________________________
None
Train on 252 samples, validate on 6

In [ ]:
# embedding_vecor_length = 32
# model2 = Sequential()
# model2.add(Embedding(vocab_size, embedding_vecor_length,input_length=max_length))
# from keras.layers import LSTM
# model2.add(LSTM(100))
# model2.add(Dense(12, activation='sigmoid'))
# model2.add(Dense(12, activation='sigmoid'))
# model2.add(Dense(12, activation='sigmoid'))
# model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# print(model.summary())
# model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=64)

In [ ]:
# from keras.layers import Dropout
# embedding_vecor_length = 32
# model3 = Sequential()
# model3.add(Embedding(vocab_size, embedding_vecor_length,input_length=max_length))

# model3.add(Dense(12, activation='sigmoid')) 
# model3.add(Dropout(0.3))
# model3.add(Dense(12, activation='sigmoid'))
# model3.add(Dropout(0.3))
# model3.add(Dense(12, activation='sigmoid'))
# model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# print(model.summary())
# model3.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=64)

In [ ]:
def create_model():
    # create model
    model = Sequential()
    model.add(Embedding(vocab_size, 32, input_length=max_length))
    model.add(Flatten())
    model.add(Dense(12, activation='sigmoid'))
    
    # compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
    return model

In [ ]:
# from keras.wrappers.scikit_learn import KerasClassifier
# from sklearn.model_selection import GridSearchCV


# model = KerasClassifier(build_fn=create_model,verbose=0)
# # define the grid search parameters
# # learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
# # momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
# epochs=[1,3,5]
# batch_size=[32,64,128,256]
# param_grid = dict(epochs=epochs,batch_size=batch_size)
# grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)

# grid_result = grid.fit(X_train, y_train)
# # summarize results
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))